# Time Series Analysis for Guayas Region // Week 1 // Data Preperation

## 1.Import Libraries

In [1]:
import pandas as pd # data manupulation
import numpy as np # numeric operations
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # data visualization
import os
from google.colab import drive # Import drive folder to google colab
from pathlib import Path
import gc

## 2.Config and Loading DataFrames

### 1.Connect Google Drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


### 2.Set Data Path

In [3]:
DATA_PATH = Path("/content/drive/MyDrive/Time Series Analysis /favorita-grocery-sales-forecasting")

### 3.Data Loading

In [4]:
#Load all supporting CSV files except train.csv.
df_items = pd.read_csv(DATA_PATH / "items.csv")
df_stores = pd.read_csv(DATA_PATH / "stores.csv")
df_oil = pd.read_csv(DATA_PATH / "oil.csv")
df_holidays_events = pd.read_csv(DATA_PATH / "holidays_events.csv")
df_transactions = pd.read_csv(DATA_PATH / "transactions.csv")

In [ ]:
 ###Load train.csv in chunks and filter only for given store_ids.
guayas_stores = df_stores.loc[df_stores["state"] == "Guayas", "store_nbr"].unique()
chunks = []
for chunk in pd.read_csv(
    DATA_PATH / "train.csv",
    parse_dates=["date"],
    chunksize=1_000_000,):
    chunks.append(chunk[chunk["store_nbr"].isin(guayas_stores)])

df_train = pd.concat(chunks, ignore_index=True)

del chunks


/tmp/ipython-input-358988839.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(


In [ ]:
df_train.shape

### 4.Reduce Data by Top 3 Product Families

In [ ]:
# Compute the number of items per family and select the top 3 families
item_counts = df_items.groupby("family")["item_nbr"].nunique().sort_values(ascending=False)
top_families = item_counts.head(3)


# Get only items that belong to the top families
items_top = df_items[df_items["family"].isin(top_families.index)]  # use .index (family names)

# Keep only training data with those items
df_train = df_train[df_train['item_nbr'].isin(items_top["item_nbr"])]



In [ ]:
# Plot the top 3 families
plt.figure(figsize=(8, 5))
plt.bar(top_families.index, top_families.values, color='skyblue', edgecolor='black')  # Use .index and .values
plt.title("Top 3 Families by Number of Items", fontsize=20, fontweight='bold')
plt.xlabel("Family", fontsize=16)
plt.ylabel("Number of Items", fontsize=16)
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
df_train.head()

### 6.Down-sample for speedy experiments

In [ ]:
df_train.shape

In [ ]:
df_train = df_train.sample(n=2000000, random_state=42)

df_train.shape

### 7. Save a .Pickle file

In [ ]:
#Save DataFrame to drive folder as .pickle file
save_path = '/content/drive/MyDrive/Time Series Analysis /favorita-grocery-sales-forecasting/df_train_filtered.pkl'

df_train.to_pickle(save_path)

df_train.shape

In [ ]:
# load df_train dataframe from pickle.file to reload without rerunning the chunk loop again
df_train = pd.read_pickle(DATA_PATH / "df_train_filtered.pkl")

In [ ]:
df_train.head()

## 3.Missing Value Check

In [ ]:
dfs = {
    "stores": df_stores,
    "items": df_items,
    "transactions": df_transactions,
    "holidays_events": df_holidays_events,
    "oil": df_oil,
    "train": df_train
}

for name, df in dfs.items():
    print("\nMissing values in", name)
    print(df.isnull().sum()[df.isnull().sum() > 0])

## 4.Data Quality Fixes

### 1.Fill onpromotion Nas with False

In [ ]:
#Fill onpromotion NAs with False
df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)



In [ ]:
# Replace False and True by 0 and 1
df_train['onpromotion'] = df_train['onpromotion'].apply(lambda x: 1 if x == True else 0)

### 2.Interpolate Oil Price

In [ ]:
 # Interpolate oil price
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate().fillna(method='bfill')

### 3.Fill Missing Transactions with 0

In [ ]:
# Fill missing transactions with 0
df_transactions["transactions"] = df_transactions["transactions"].fillna(0)

### 4.Adjust negative Sales to 0

In [ ]:
# Checking for negative sales (returns)
negative_sales = df_train[df_train['unit_sales'] < 0]

negative_sales.head()  # Viewing negative sales for analysis

In [ ]:
# Replacing negative sales with 0 to reflect returns as non-sales
df_train['unit_sales'] = df_train['unit_sales'].apply(lambda x: max(x, 0))

### 5.Filling Missing Calender Days

In [ ]:
# convert date to datetime
df_train['date'] = pd.to_datetime(df_train['date'])

def fill_calendar(group):
    # group contains all rows for ONE (store_nbr, item_nbr) pair
    g = group.set_index("date").sort_index()   # use calendar as the index
    g = g.asfreq("D", fill_value=0)            # make it daily; add 0 where missing
                                                # put the identifiers back (asfreq drops them)
    g["store_nbr"] = group["store_nbr"].iloc[0]
    g["item_nbr"]  = group["item_nbr"].iloc[0]

    return g.reset_index()                     # date back to a normal column

df_train = (
    df_train
    .groupby(["store_nbr", "item_nbr"], group_keys=False)  # keeps memory low
    .apply(fill_calendar)
)

df_train.head()

## 5.Outlier Handling

### 1.Z-Score Calculation and Outlier Detection

In [ ]:
# Function to calculate Z-score for each group (store-item combination)
def calculate_store_item_zscore(group):
    # Compute mean and standard deviation for each store-item group
    mean_sales = group['unit_sales'].mean()
    std_sales = group['unit_sales'].std()

    # Calculate Z-score for unit_sales (avoiding division by zero for standard deviation), and store it in a new column called z_score
    group['z_score'] = (group['unit_sales'] - mean_sales) / (std_sales if std_sales != 0 else 1)
    return group

# Apply the Z-score calculation to each store-item group, then flatten the index
df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_store_item_zscore)
df_train_grouped.reset_index(drop=True, inplace=True)

# Define threshold for outliers (e.g., Z-score > 5)
outliers = df_train_grouped[df_train_grouped['z_score'] > 5]

# Print summary
print(f"Number of outliers detected: {len(outliers)}")
outliers.head()

In [ ]:
df_train.shape


## 5.Feature Engineering

### 1.Date Splitting

In [ ]:
#Extrat date by year, month, day and day of the week
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.dayofweek
df_train['is_weekend'] = df_train['day_of_week'].isin([5, 6])

### 2.Rolling Means

In [ ]:
# Calculating rolling average of unit_sales
# a 7-day average puts weekday and weekend sales on equal footing
df_train['unit_sales_7d_avg'] = df_train.groupby(['item_nbr', 'store_nbr'])['unit_sales'].transform(lambda x: x.rolling(window=7).mean())

### 3.Lag Features

In [ ]:
# Create lag features for sales data:
# for short-term,weekly and monthly patterns in sales behavior
df_train['lag_1'] = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(1) #short-term
df_train['lag_7'] = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(7) #weekly
df_train['lag_30'] = df_train.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(30) #monthly

### 4.Merge Holidays

In [ ]:
# Process the holiday merge in chunks to avoid memory overload
chunk_size = 1000000
chunks = []

df_holidays_events['date'] = pd.to_datetime(df_holidays_events['date'])
holidays_dict = df_holidays_events.set_index('date')['type'].to_dict()

for i in range(0, len(df_train), chunk_size):
    chunk = df_train.iloc[i:i+chunk_size].copy()

    # Map holiday information using the dictionary
    chunk['holiday_type'] = chunk['date'].map(holidays_dict)
    chunk['is_holiday'] = chunk['holiday_type'].notna()

    chunks.append(chunk)

    # Clear memory
    del chunk
    gc.collect()

# Concatenate all chunks
df_train = pd.concat(chunks, ignore_index=True)


### 5.Sales Growth & Change Rate

In [ ]:
# Measures momentum (for promotions and seasonality)
df_train["sales_change_7d"] = (
    df_train.groupby(["store_nbr", "item_nbr"])["unit_sales"]
    .transform(lambda x: x.pct_change(periods=7))
)

### 6.Economics Influence from Oil Prices

In [ ]:
# merge oil to capture macroeconomic effects
#rooling oul average to smooth volatility
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil["rolling_oil_7d"] = df_oil["dcoilwtico"].rolling(7, min_periods=1).mean()
df_train = df_train.merge(df_oil, on="date", how="left")

In [ ]:
df_train.head()

## 6.Exploratory Data Analysis

### 1.Trend Plot

In [ ]:
# Aggregating total sales by date
sales_by_date = df_train.groupby('date')['unit_sales'].sum()

# Plotting the time-series
plt.figure(figsize=(12,6))
plt.plot(sales_by_date.index, sales_by_date.values)
plt.title('Total Unit Sales Over Time in Guayas state', fontsize=20, fontweight='bold')
plt.xlabel('Date', fontsize=16)
plt.ylabel('Unit Sales', fontsize=16)
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)
plt.show()

### 2.Monthly Heatmap

In [ ]:
# Aggregating sales by year and month
sales_by_month = df_train.groupby(['year', 'month'])['unit_sales'].sum().unstack()


plt.figure(figsize=(8, 5))  # Increase figure size for better visibility
sns.heatmap(
    sales_by_month,
    cmap='coolwarm',  # Use a diverging colormap for better contrast
    linewidths=0.5,  # Add lines between cells for clarity
    linecolor='white',  # Use white lines for a cleaner look
    cbar_kws={'label': 'Sales Volume'}  # Add a descriptive colorbar label
)

# Customizing title and axes labels
plt.title('Monthly Sales Trends Over Years', fontsize=22, fontweight='bold')
plt.xlabel('Month', fontsize=18, labelpad=10)  # Labelpad adds spacing
plt.ylabel('Year', fontsize=18, labelpad=10)

# Formatting tick labels
plt.xticks(fontsize=14, rotation=45)  # Rotate x-axis labels for better readability
plt.yticks(fontsize=14)

# Adjust layout for better spacing
plt.tight_layout()

# Display the heatmap
plt.show()

In [ ]:
df_train[(df_train['year'] == 2017) & (df_train['month'] == 8) & (df_train['day']>=16)]

The Anomalie of low sale trend in the last month (August 2017) occurs  because the sales data ends on 15-08-2017. There are a lack of data for the half month

### 3.Promotion Impact

In [ ]:
#Promotion Impact - Compare sales during promotion vs. non-promotion periods.
promo_avg = df_train.groupby("onpromotion")["unit_sales"].mean()
promo_avg.plot(kind="bar", title="Promotion vs Non-Promotion Sales")

### 4.Holiday Impact

In [ ]:
#the impact of holidays exclude day with zero sales like it was at the very beginning
df_train_no_zero = df_train[df_train.unit_sales > 0]


In [ ]:
df_train_no_zero['date'] = pd.to_datetime(df_train_no_zero['date'])
df_holidays_events['date'] = pd.to_datetime(df_holidays_events['date'])

In [ ]:
# Merging df_train with zero sales with data with holidays
df_train_holiday = pd.merge(df_train_no_zero, df_holidays_events, on='date', how='left')

In [ ]:
# Aggregating sales by holiday and non-holiday
holiday_sales = df_train_holiday.groupby('type')['unit_sales'].mean()

# Plotting holiday impact
plt.figure(figsize=(8,5))
holiday_sales.plot(kind='bar', color='lightgreen', edgecolor='black')
plt.title('Impact of Holidays on Sales', fontsize=20, fontweight='bold')
plt.ylabel('Average Unit Sales', fontsize=16)
plt.xlabel('')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
# impact of Holidays
nonzero_sales = df_train[df_train['unit_sales'] > 0]
holiday_avg = nonzero_sales.groupby('is_holiday')['unit_sales'].mean()
holiday_avg.plot(kind='bar', color='lightgreen', edgecolor='black', title="Holiday Impact on Sales")
plt.show()

### 5.Perishable Share

In [ ]:
# Measures momentum (for promotions and seasonality)
df_train["sales_change_7d"] = (
    df_train.groupby(["store_nbr", "item_nbr"])["unit_sales"]
    .transform(lambda x: x.pct_change(periods=7)))

### 6.Economic Influence from Oil Prices

In [ ]:
# Merging df_train with items to get perishable data
df_items['perishable']= df_items['perishable'].astype(bool)
df_train_items = pd.merge(df_train, df_items, on='item_nbr', how='left')
df_train_items.head()

In [ ]:
df_train_items['perishable'].value_counts()

## 7.Save Prepared Dataset

In [ ]:
# Filter DataFrame by date range
min_date = '2014-01-01'
max_date = '2014-04-01'

df_train_filter = df_train_items[
    (df_train_items['date'] >= min_date) &
    (df_train_items['date'] < max_date)
]

# Define Google Drive folder path
drive_folder_path = '/content/drive/MyDrive/Time Series Analysis /favorita-grocery-sales-forecasting'

# Save the CSV file
output_file_path = f"{drive_folder_path}guayas_dataset.csv"
df_train_filter.to_csv(output_file_path, index=False)
